# LSTM Model Type 3 Age

In [1]:
import pandas as pd
import numpy as np
from numpy import array
from collections import defaultdict

import keras_tuner
from keras.layers import LSTM
from keras.layers import Dense
from keras.models import Sequential
import tensorflow as tf

from LSTM_Model_Type3 import split_position, LSTM_FitPredict

### Set Random Seed for Code Reproductability

In [2]:
import random
random.seed(2020)

from numpy.random import seed
seed(2020)
tf.random.set_seed(2020)

import random as python_random
python_random.seed(2020)

### Read in Raw Data & Preprocessing

In [3]:
# Read in the raw age-sex cohort data without the remainder area
# Prepare the unique sa3 names, codes for LSTM fitting selection
raw_data = pd.read_csv('../Data/true_1000_fulldata.csv')
sa3_num = len(raw_data['SA3 Code'].unique())
sa3_names = raw_data['SA3 Name'].unique()
sa3_codes = raw_data['SA3 Code'].unique()

### Preprocessing by Splitting Population into Sex-Group

In [4]:
# Split the data by sex
age_groups = ['0-4','5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39','40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74','75-79', '80-84', '85+']
male_age_groups = ['m0-4', 'm5-9', 'm10-14', 'm15-19', 'm20-24', 'm25-29', 'm30-34', 'm35-39', 'm40-44', 'm45-49', 'm50-54', 'm55-59', 'm60-64', 'm65-69', 'm70-74', 'm75-79', 'm80-84', 'm85+']
female_age_groups = ['f0-4', 'f5-9', 'f10-14', 'f15-19', 'f20-24', 'f25-29', 'f30-34', 'f35-39', 'f40-44', 'f45-49', 'f50-54', 'f55-59', 'f60-64', 'f65-69', 'f70-74', 'f75-79', 'f80-84', 'f85+']
population_m_dict = defaultdict(dict)
population_f_dict = defaultdict(dict)

# save population for each year into lists
for sa3_code in sa3_codes:
    population_m_dict[sa3_code] = dict()
    population_f_dict[sa3_code] = dict()
    for year in range(1991,2012):
        if(raw_data[(raw_data['Year']==year) & (raw_data['SA3 Code']==sa3_code)]['Total'].size>0):
            population_m_dict[sa3_code][year] = raw_data[(raw_data['Year']==year) & (raw_data['SA3 Code']==sa3_code)][male_age_groups].values.tolist()[0]
            population_f_dict[sa3_code][year] = raw_data[(raw_data['Year']==year) & (raw_data['SA3 Code']==sa3_code)][female_age_groups].values.tolist()[0]

### Create Dataframe for Storing Prediction Result

In [5]:
output = pd.DataFrame(index = range(sa3_num*36), columns = ['Code','Area name','Sex','Age group',2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011])
for sa3_code in range(0,sa3_num):
    for age_group in range(18):
        output.loc[sa3_code*36+age_group] = {'Code':sa3_codes[sa3_code],'Area name':sa3_names[sa3_code],'Sex':'Females','Age group':age_groups[age_group]}
        output.loc[sa3_code*36+18+age_group] = {'Code':sa3_codes[sa3_code],'Area name':sa3_names[sa3_code],'Sex':'Males','Age group':age_groups[age_group]}

### Start LSTM Fitting & Predicting

##### Define Variable for Splitting the Full Dataset

In [6]:
# Key Year for Model Fitting
train_start = 1991
train_end = 2001

# choose a number of time steps
n_steps = 1

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
n_age_groups = 1

# Number of Epoch(es)
epoch = 2

# Obtain the split position for the training sex, validation sex, and the first input_x
train_val_bounds, test_bounds = split_position(n_steps, train_start, train_end)

##### LSTM Model Defining

In [7]:
'''Function for Defining the LSTM Model'''
def lstm_model(lstm1_units, activ, optimizer, loss_fun):
    model = Sequential()
    model.add(LSTM(lstm1_units, activation=activ, input_shape=(n_steps, n_features)))
    model.add(Dense(n_age_groups))
    model.compile(optimizer=optimizer, loss=loss_fun, metrics=["mean_absolute_percentage_error"])
    return model

##### LSTM Model Tunning

In [8]:
# Define HYPERPARAMETERS
lstm1_min = 100
lstm1_max = 1000
lstm1_step = 100
activ_functions = ["relu"]
optimizers = ["adam","adagrad"]
loss_functions = ["mse"]

'''In-Build Function for LSTM Model Tuning'''
def build_model(hp):
    lstm1_units = hp.Int("LSTM units", min_value=lstm1_min,max_value=lstm1_max,step=lstm1_step)
    activation = hp.Choice("Activation Function", activ_functions)
    optimizer = hp.Choice("Optimizer", optimizers)
    loss_fun = hp.Choice("Loss Function", loss_functions)
    model = lstm_model(lstm1_units, activation, optimizer, loss_fun)
    return model

# Generate the tuned model with keras_tuner package
tuner = keras_tuner.RandomSearch(hypermodel=build_model, objective=keras_tuner.Objective("val_mean_absolute_percentage_error", "min"))

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
Metal device set to: Apple M1 Pro
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


2022-09-01 19:02:49.526711: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-01 19:02:49.526810: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


##### Generate Prediction Result

In [ ]:
# Generate each sex's Prediction Result of each age-cohort for all areas and store them into csv
output = LSTM_FitPredict(sa3_codes, population_m_dict, male_age_groups, n_steps, train_val_bounds, test_bounds, n_features, epoch, "Males", 2002, tuner, output)
output = LSTM_FitPredict(sa3_codes, population_f_dict, female_age_groups, n_steps, train_val_bounds, test_bounds, n_features, epoch, "Females", 2002, tuner, output)
output.to_csv('../Data/Projection/Iterative_predict_output_Step1_1000_Type2.csv',index=False,header=True)

INFO:tensorflow:Oracle triggered exit


2022-09-01 19:02:49.904943: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-01 19:02:50.296643: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-01 19:02:50.606893: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-01 19:02:50.779268: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-01 19:02:50.889785: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2022-09-01 19:02:52.024924: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-01 19:02:52.301495: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2022-09-01 19:02:53.324738: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-01 19:02:53.595812: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Oracle triggered exit


...

2022-09-01 23:20:24.623055: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-01 23:20:26.876784: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-01 23:20:28.597893: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-01 23:20:29.274148: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Oracle triggered exit


2022-09-01 23:20:32.344343: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-01 23:20:34.584138: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-01 23:20:36.345024: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-01 23:20:37.021643: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Oracle triggered exit


2022-09-01 23:20:40.289283: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-01 23:20:42.569031: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
